In [2]:
from collections import defaultdict, deque
import random
import math
import time

import numpy as np

class City:
    id = 0
    def __init__(self):
        self.X = random.randint(-100, 100)
        self.Y = random.randint(-100, 100)
        self.Z = random.randint(0, 50)
        self.coordinates = np.array((self.X, self.Y, self.Z))
        self.id = City.id
        City.id += 1
    
class Graph:
    
    def __init__(self, cities):
        self.all_paths = []
        self.distance_list = []
        self.adjList = defaultdict(list)
        self.sym_matrix = self.generate_sym_matrix(cities)
        self.matrix_80 = self.generate_80_matrix(cities)
        self.asym100_matrix = self.generate_asym_matrix(cities)
        self.asym80_matrix = self.generate_80asym_matrix(cities)

    #Stworzenie symetrycznej macierzy 100%

    def generate_sym_matrix(self, cities):
        sym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i != j:
                    row.append(1)
                else:
                    row.append(0)
            sym_matrix.append(row)
        return np.array(sym_matrix)
    

    #Stworzenie symetrycznej macierzy 80%

    def generate_80_matrix(self, cities):
        matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i != j:
                    row.append(1)
                else:
                    row.append(0)
            matrix.append(row)
        matrix = np.array(matrix)
        total_elements = matrix.size
        num_to_change = int(0.2 * total_elements)
    
        indices_to_change = np.random.choice(total_elements, num_to_change, replace=False)
        matrix_flattened = matrix.flatten()
    
        for index in indices_to_change:
            if matrix_flattened[index] == 1:
                matrix_flattened[index] = 0
    
        return matrix_flattened.reshape(matrix.shape)
    

    #Stworzenie asymetrycznej macierzy 100%

    def generate_asym_matrix(self, cities):
        asym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i == j:
                    row.append(0)
                elif cities[i].Z < cities[j].Z and i != j:
                    row.append(2)
                elif cities[i].Z >= cities[j].Z and i != j:
                    row.append(1)
            asym_matrix.append(row)
        return np.array(asym_matrix)
    

    #Stworzenie asymetrycznej macierzy 80%

    def generate_80asym_matrix(self, cities):
        asym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i == j:
                    row.append(0)
                elif cities[i].Z < cities[j].Z and i != j:
                    row.append(2)
                elif cities[i].Z >= cities[j].Z and i != j:
                    row.append(1)
            asym_matrix.append(row)
        asym_matrix = np.array(asym_matrix)
        total_elements = asym_matrix.size
        num_to_change = int(0.2 * total_elements)
    
        indices_to_change = np.random.choice(total_elements, num_to_change, replace=False)
        matrix_flattened = asym_matrix.flatten()
    
        for index in indices_to_change:
            if matrix_flattened[index] == 1 or matrix_flattened[index] == 2:
                matrix_flattened[index] = 0
    
        return matrix_flattened.reshape(asym_matrix.shape)


    #Metoda obliczajaca odleglosc bez uwzglednienia asymetrycznosci
    def calculate_distance(self, city1, city2):
        return np.linalg.norm(city1.coordinates - city2.coordinates)
    


    #Metoda obliczajaca odleglosc z uwzglednieniem asymetrycznosci
    def calculate_distance_asym(self, city1, city2):
        if city1.Z < city2.Z:
            #print("Going up")
            return 1.1 * math.sqrt((city2.X - city1.X) ** 2 + (city2.Y - city1.Y) ** 2 + (city2.Z - city1.Z) ** 2)
        elif city1.Z > city2.Z:
            #print("Going down")
            return 0.9 * math.sqrt((city2.X - city1.X) ** 2 + (city2.Y - city1.Y) ** 2 + (city2.Z - city1.Z) ** 2)
        else:
            #print("Same height")
            return np.linalg.norm(city1.coordinates - city2.coordinates)


    #Dodawanie krawedzi do grafu (do zmiennej adjList)
    def addEdge(self, u, v):
            self.adjList[u].append(v)

    #Definiowanie krawedzi i przekazanie do metody addEdge
    def createEdge(self, matrix):
        for i in range(0, len(matrix)):
            for j in range(0, len(matrix)):
                if matrix[i][j] == 1 or matrix[i][j] == 2:
                    self.addEdge(i, j)
                    #print(f"Created edge between {i} and {j}")

    def heuristic(self, cities, sym):
        h_dop = float('inf') #Min odlegosc
        h_niedop = 0 #Srednia odleglosc
    
        current_dist = 0
        divide = 0
        for i in range(len(cities)):
            for j in self.adjList[i]:
                if sym == 'sym':
                    current_dist = self.calculate_distance(cities[i], cities[j])
                elif sym == 'asym':
                    current_dist = self.calculate_distance_asym(cities[i], cities[j])
                h_niedop += current_dist
                # print(f'Current distance between {i} and {j}: {current_dist}')

                divide += 1

                if current_dist < h_dop:
                    # print(f'{current_dist} is lower than {h_dop}')
                    h_dop = current_dist
        
        h_niedop = h_niedop / divide
        # print(f'h_niedop = {h_niedop}, h_dop = {h_dop}')
        return h_dop, h_niedop
                
    #Metoda obliczajaca najkrotsza trase metoda A*
    def astar(self, startNode, cities, sym):
        h_dop, h_niedop = self.heuristic(cities, sym)
        queue = deque([(0, startNode, [startNode], 0)])
        paths = []
        h_chosen = h_dop
        total_distance = 0
        print(f'\nA*\nHeuristic: {h_chosen}')

        while queue:
            # print(f'\nBefore: {queue}')
            queue = deque(sorted(queue, key=lambda row: row[0]))
            # print(f'After: {queue}')
            total_cost, currentNode, path, distance_passed = queue.popleft()
            # print(f'Popped cost: {total_cost}, path: {path}\n')

            # print(path)
            
            if len(path) == len(self.adjList):
                paths.append(path)
                # print(path)
                continue

            for neighbour in self.adjList[currentNode]:
                if neighbour not in path:
                    if sym == 'sym':
                        actual_cost = self.calculate_distance(cities[currentNode], cities[neighbour])
                    elif sym == 'asym':
                        actual_cost = self.calculate_distance_asym(cities[currentNode], cities[neighbour])
                    g_n = distance_passed + actual_cost
                    h_n = (6-len(path)) * h_chosen
                    # print(f'Heuristic from {neighbour}: {h_n}, G_n cost: {g_n}, toal cost: {total_cost}, actual cost: {actual_cost}')
                    f_cost = g_n + h_n
                    queue.append((f_cost, neighbour, path + [neighbour], g_n))
                elif len(path) == len(self.adjList) - 1 and neighbour == 0:
                    path.append(0)
                    paths.append(path)
                    print(path)
                    queue.clear()
        paths = paths[0]
        for i in range(0, len(paths) - 1):
            if sym == 'sym':
                total_distance += self.calculate_distance(cities[paths[i]], cities[paths[i + 1]])
            elif sym == 'asym':
                total_distance += self.calculate_distance_asym(cities[paths[i]], cities[paths[i + 1]])
        return paths, total_distance

In [3]:
cities = []
for i in range(0, 6):
    cities.append(City())
    # print(f'City {i}: X: {cities[i].X}, Y: {cities[i].Y}, Z: {cities[i].Z}')
graph = Graph(cities)

#print(graph.sym_matrix)
#print(graph.matrix_80)
# print(graph.asym100_matrix)
#print(graph.asym80_matrix)

#graph.createEdge(graph.sym_matrix)
# graph.createEdge(graph.matrix_80)
# graph.createEdge(graph.asym100_matrix)
graph.createEdge(graph.asym80_matrix)



start_time = time.perf_counter()
best_path, best_distance = graph.astar(0, cities, 'asym')
print(f'Best path: {best_path}, best dist: {best_distance}')
end_time = time.perf_counter()
print(f'A* time: {end_time - start_time}')


A*
Heuristic: 30.146641603999605
[0, 3, 4, 2, 1, 0]
Best path: [0, 3, 4, 2, 1, 0], best dist: 446.8350394903996
A* time: 0.0006629999843426049
